### Extracting Data From xml File

In [13]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
tree = ET.parse('train_data.xml')
reviews = tree.getroot()




Intialisations

In [14]:
Sentence_array=list()
asp_array=list()
ref_revarray=list()
effect_array=list()
revindex=list()
i=0
j=0
prev=0
num_sent=0

polar = {
    "negative": 0.0,
    "neutral":0.5,
    "positive":1.0
}

In [15]:
for review in reviews:

    if(i==299):
        ref_revarray.append(i)
        num_sent+=1
        effect_array.append(0.0)
        asp_array.append('FOOD#QUALITY')
        Sentence_array.append("Food is very bad")

    sentences = review.find("./sentences")
    for sentence in sentences :
        num_sent=num_sent+1
        text = sentence.find("./text")
        opinions = sentence.findall("./Opinions/Opinion")
        for opinion in opinions :
            if(prev == i):
                j=j+1
            else:
                revindex.append(j)
                j=1
                prev=i

            effect_array.append(polar[opinion.attrib['polarity']])
            Sentence_array.append(text.text)
            asp_array.append(opinion.attrib['category'])
            ref_revarray.append(i)

    i=i+1

revindex=np.array(revindex)
pol_arr=np.array(effect_array)

maxi=np.max(revindex)

print("Total number of sentences are " + str(len(Sentence_array)))
print("Maximim number of sentences in a review is " + str(maxi))



print(num_sent)  
print(len(Sentence_array))
print(len(asp_array))
print(len(ref_revarray))
print(len(effect_array))




Total number of sentences are 2508
Maximim number of sentences in a review is 44
2001
2508
2508
2508
2508


### Creating DataFrame

In [16]:
data=list()

for i in range(len(Sentence_array)):
  data.append([ref_revarray[i], asp_array[i],Sentence_array[i],effect_array[i]])
  
# Create the pandas DataFrame
DATA = pd.DataFrame(data, columns = ['rid','aspect','text','polarity'])

DATA[0:150]


,rid,aspect,text,polarity
0,0,RESTAURANT#GENERAL,Judging from previous posts this used to be a ...,0.0
1,0,SERVICE#GENERAL,"We, there were four of us, arrived at noon - t...",0.0
2,0,SERVICE#GENERAL,"They never brought us complimentary noodles, i...",0.0
3,0,FOOD#QUALITY,The food was lousy - too sweet or too salty an...,0.0
4,0,FOOD#STYLE_OPTIONS,The food was lousy - too sweet or too salty an...,0.0
...,...,...,...,...
145,22,AMBIENCE#GENERAL,"Leon is an East Village gem: casual but hip, w...",1.0
146,22,FOOD#QUALITY,"Leon is an East Village gem: casual but hip, w...",1.0
147,22,AMBIENCE#GENERAL,"Leon is an East Village gem: casual but hip, w...",1.0
148,22,FOOD#QUALITY,"Leon is an East Village gem: casual but hip, w...",1.0


In [17]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))


def rem_stopwords(text):
    text1 = [w for w in text.split() if not w.lower() in stop_words]
    return " ".join(text1)

def tokenise_data(samp_text):
    word_tokens = [word.lower() for word in word_tokenize(samp_text)]
    return word_tokens
    



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sriramprasad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
import re
import string


def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)


def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

def remove_punct(text):
  table = str.maketrans("", "", string.punctuation)
  return text.translate(table)



In [19]:
import sys
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install tensorflow

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [20]:
DATA.text = DATA.text.map(lambda x: remove_URL(x))
DATA.text = DATA.text.map(lambda x: remove_html(x))
DATA.text = DATA.text.map(lambda x: remove_emoji(x))
DATA.text = DATA.text.map(lambda x: remove_punct(x))
DATA.text = DATA.text.map(rem_stopwords)

hell=list()

corpus=list()
size=list()


hell = DATA.loc[:,"text"].tolist()
hell1 = DATA.loc[:,"rid"].tolist()

for i in range(len(Sentence_array)):
    corpus.append(tokenise_data(hell[i]))
    size.append(len(corpus[i]))


corpus=np.array(corpus)
hell1 = np.array(hell1)
size=np.array(size)

maximum= np.max(size)
maximum1=np.max(hell1)
print("maximum words in a sentence are " + str(maximum))
print("maximum sentences in a review are" +str(maxi))
print("Total Number Of Reviews",+maximum1+1)


maximum words in a sentence are 35
maximum sentences in a review are44
Total Number Of Reviews 350


/tmp/ipykernel_3703/2397471873.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  corpus=np.array(corpus)


### Padding And Final Test And Train Data

In [21]:
tokenizer = Tokenizer(num_words=1000000)
tokenizer.fit_on_texts(DATA.text)

sequences = tokenizer.texts_to_sequences(DATA.text)

paddedsequences = pad_sequences(
sequences, maxlen=35, truncating="post", padding="post"
)

print(paddedsequences)
print(paddedsequences)
print(type(paddedsequences[0]))
print(len(paddedsequences[0]))


[[2058  609 2059 ...    0    0    0]
 [ 202   34  516 ...    0    0    0]
 [  17  611   34 ...    0    0    0]
 ...
 [ 605  347  292 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]]
[[2058  609 2059 ...    0    0    0]
 [ 202   34  516 ...    0    0    0]
 [  17  611   34 ...    0    0    0]
 ...
 [ 605  347  292 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]]
<class 'numpy.ndarray'>
35


In [22]:
final_train_seq=paddedsequences[:2010]
final_test_seq=paddedsequences[2010:]

final_train_label=pol_arr[:2010]
final_test_label=pol_arr[2010:]

print(final_train_seq.shape)

print(final_test_seq)


(2010, 35)
[[2821  647    0 ...    0    0    0]
 [   2    1    0 ...    0    0    0]
 [  51  396  519 ...    0    0    0]
 ...
 [ 605  347  292 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]
 [ 249   91    7 ...    0    0    0]]


### Fittinf model and caluclating Accuracies

In [24]:

from keras.models import Sequential
from keras.layers import Dense,Embedding,SimpleRNN
from keras import losses

model= Sequential()
model.add( Embedding(30000,1,input_length = 35))#embeddings dimension as 32
model.add(SimpleRNN(16,input_shape = (2009, 35), return_sequences=False,activation="relu"))
model.add(Dense(1, activation='sigmoid'))#Dense layer of 10 units

model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])



2022-12-02 21:33:08.155324: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-02 21:33:08.155368: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sriram): /proc/driver/nvidia/version does not exist
2022-12-02 21:33:08.155854: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
history = model.fit(final_train_seq,final_train_label,validation_data = (final_test_seq,final_test_label),epochs = 5 ,batch_size= 128,verbose = 1)

Epoch 1/5
16/16 [==============================] - 3s 41ms/step - loss: 0.6871 - accuracy: 0.6632 - val_loss: 0.6971 - val_accuracy: 0.4197
Epoch 2/5
16/16 [==============================] - 0s 13ms/step - loss: 0.6707 - accuracy: 0.7204 - val_loss: 0.7032 - val_accuracy: 0.4197
Epoch 3/5
16/16 [==============================] - 0s 14ms/step - loss: 0.6531 - accuracy: 0.7204 - val_loss: 0.7134 - val_accuracy: 0.4197
Epoch 4/5
16/16 [==============================] - 0s 13ms/step - loss: 0.6324 - accuracy: 0.7204 - val_loss: 0.7315 - val_accuracy: 0.4197
Epoch 5/5
16/16 [==============================] - 0s 14ms/step - loss: 0.6082 - accuracy: 0.7204 - val_loss: 0.7694 - val_accuracy: 0.4197


In [ ]:
prediction=model.predict(final_test_seq)
final_pred=list()

for i in range(len(prediction)):
    if(prediction[i] < 0.5):
        final_pred.append(0.0)

    else:
        final_pred.append(1.0)


final_pred=np.array(final_pred)
